<a href="https://colab.research.google.com/github/EvaMare/Foundations-of-Machine-Learning/blob/main/Large_Scale_Modelling_Tutorial_DCBT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Large-Scale Modeling of Brain Dynamics
## DCBT 2022

__Content creator:__ Francisco Santos (f.pascoadossantos@gmail.com)


# Introduction

Computational modeling is a powerful tool for the exploration of neural dynamics, allowing for a more direct analysis and easier parameter manipulation than physiological studies. One can achieve this by applying mathematic concepts to the description neural dynamics at different scales, from the molecular and synaptic level to modeling of large-scale activity. The later relies on the assumption that the activities of individual neurons are generally uncorrelated, meaning that the ensemble activity can be reduced to its mean and variance. This mean-field approach can be extremely useful in computational neuroscience by allowing for the modeling of large-scale dynamics at a higher level of abstraction which is more computationally viable and tractable. That said, in the past years, large-scale modeling of the human brain has helped advance our knowledge about how cortical areas interact to form networks that are important for a range of cognitive processes.
A comprehensive review on large-scale neural models can be consulted in https://www.nature.com/articles/nn.4497.

$$$$

In this tutorial, we will learn the fundamentals of neural-mass models, how to mainpulate their parameters to show different behaviors and how to simulate neural dynamics with networks of coupled neural masses. In addition, we will look at human whole-brain models, where nodes are connected with structural data from the human brain.
Finally, we will generate BOLD timeseris from our model and calculate functional connectivity. We will also compare functional connectivity in a healthy vs lesioned connectome.

This notebook is meant to accompany a Gala Tutorial, which can be consulted at: https://www.learngala.com/cases/large-scale-brain-dynamics-modelling.
In addition, the slides used for the DCBT2022 tutorial, will be available upon request.

In [ ]:
# @title Run this cell to import required modules

!pip install networkx --quiet
!pip install tqdm --quiet
!pip install nilearn --quiet

import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
import networkx as nx
import gdown
import ipywidgets as widgets

from google.colab import files
from tqdm.notebook import tqdm
from sklearn.linear_model import LinearRegression
from nilearn import plotting

In [ ]:
# @title Functions

# Auxiliary Functions

def load_connectivity():

  ## Connectivity matrix ##

  # Define the remote file to retrieve
  remote_url = 'https://drive.google.com/uc?id=1tkwQbpl0-PYwasci1Z_ClRWwe7ZlM1vz'
  # Define the local filename to save data
  local_file = 'SC_AAL78.npy'
  # Make http request for remote file data
  gdown.download(remote_url, local_file, quiet = True)

  # Gets connectivity information
  connect = np.load('SC_AAL78.npy')

  ## Coordinates ##

  # Define the remote file to retrieve
  remote_url = 'https://drive.google.com/uc?id=1U-sUjSEt2e3cCz6VWnNbccDVv_aBYRkJ'
  # Define the local filename to save data
  local_file = 'AAL78_coords.npy'
  # Make http request for remote file data
  gdown.download(remote_url, local_file, quiet = True)

  # Gets coordinate information
  coords = np.load('AAL78_coords.npy')

  ## Connectome labels ##

  # Define the remote file to retrieve
  remote_url = 'https://drive.google.com/uc?id=1E_s6tNXlKSfkD2gTD4cQAhSxl6LjM5nj'
  # Define the local filename to save data
  local_file = 'labels.npy'
  # Make http request for remote file data
  gdown.download(remote_url, local_file, quiet = True)

  # Gets coordinate information
  labels = np.load('labels.npy')

  # This sorts connectivity so that nodes are grouped by hemisphere
  order = np.concatenate((np.arange(0, 78, 2), np.arange(1, 78, 2)))

  connect = np.array([[connect[i][j] for j in order] for i in order])
  coords = np.array([coords[i] for i in order])
  labels = np.array([labels[i] for i in order])

  return connect, coords, labels

def default_parameters(**kwargs):
  pars = {}

  # Activation function parameters
  pars['F_max'] = 1 # maximum activation
  pars['mu'] = 1 # activation threshold
  pars['sigma'] = 0.25 # activation function slope

  # Time constants for both populations
  pars['tau_E'] = 2.5 # (ms) Excitatory time constant (tuned to gamma oscillations)
  pars['tau_I'] = 5 # (ms) Inhibitory time constant

  # Standard deviation of additive noise
  pars['sd_noise'] = 0.01

  # Couplings from excitatory to inhibitory population
  pars['cEE'] = 3.5 # E->E coupling
  pars['cIE'] = 3.75 # E->I coupling
  pars['cEI'] = 2.5 # I->E coupling

  # Parameter to control excitability of excitatory population
  pars['P'] = 0.31 # Defined to put the node at the transition between stable and oscillatory activity

  # External parameters if any
  pars.update(kwargs)

  return pars

def pulse(I, t_start, t_end, T, dt):

  I_vec = np.zeros(int(np.floor(T/dt)))
  I_vec[int(np.floor(t_start/dt)):int(np.floor(t_end/dt))] = I

  return I_vec

def balloon(signals, dt):

  '''
  Implementation of Balloon-Windkessel model for hemodynamics,
  as implemented by Deco et al. 2009 PNAS. Takes firing-rate activity from
  model and transforms it into a simulated BOLD fMRI signal

  Args:
    signals (array): array with shape [node, sample] containing model activity
    dt (float): time step of original signal (ms)

  Returns:
    BOLD (array): simulated BOLD signal for all nodes [node, sample]
  '''

  print('Applying Balloon-Windkessel Model...')

  BOLD = np.zeros(np.shape(signals))

  k = 0.65 # [1/s] rate of signal decay
  gamma = 0.41  # [1/s] rate of flow-dependent elimination
  tau = 0.98 # [s] hemodynamic transit time
  alpha = 0.32 # Grubbs exponent
  rho = 0.34 # resting oxygen extraction fraction
  V0 = 0.02 # resting blood volume fraction
  dt = dt / 1000 # [s]

  n = np.shape(signals)[0]

  curr_s = 1e-10 * np.ones((1, n))
  curr_f = 1e-10 * np.ones((1, n))
  curr_v = 1e-10 * np.ones((1, n))
  curr_q = 1e-10 * np.ones((1, n))

  for i in tqdm(range(1, np.shape(signals)[1])):

    new_q = curr_q + dt * (1/tau) * (curr_f/rho * (1-np.power((1-rho), (1/curr_f))) - np.power(curr_v, (1/alpha) - 1) * curr_q)

    new_v = curr_v + dt * (1/tau) * (curr_f - np.power(curr_v, 1/alpha))

    new_f = curr_f + dt * curr_s

    new_s = curr_s + dt * (np.reshape(signals[:, i-1].copy(), (1, n)) - k * curr_s - gamma*(curr_f - 1))

    y = V0 * (7*rho*(1-curr_q) + 2*(1-np.divide(curr_q, curr_v)) + (2*rho-0.2)*(1 - curr_v))

    curr_q = new_q
    curr_v = new_v
    curr_f = new_f
    curr_s = new_s

    BOLD[:, i] = np.reshape(y, (n))

  return BOLD


def bold_rs_pipeline(bold_sig, dt, fs):

  '''
  Takes np.array of BOLD signals with shape [node, step] and treats it for FC analysis.
  Pipeline based on Cabral et al. 2011 Neuroimage and Deco et al 2021 Science Advances

  Args:
    bold_sig (array): array with shape [time step, node] containing BOLD signals for all nodes
    dt (float): time step of original signal (ms)
  Returns:
    processed_sig (array): resampled bold signal
  '''

  ns = bold_sig.shape[-1]
  frs = 1/(0.72)
  fs = 1e3/dt
  nrs = int(fs/frs)
  bdr = int(60 * fs) # Time to remove at the beginning and end of signal, to avoid boundary effects from the Balloon-Windkessel model

  n_samples = int(frs * (bold_sig.shape[-1] - 2 * bdr) / fs)
  treated_sig = np.zeros((bold_sig.shape[0], n_samples))

  for n in range(bold_sig.shape[0]):
      treated_sig[n, :] = sig.resample(bold_sig[n, bdr:-bdr], n_samples)
      treated_sig[n, :] -= np.mean(treated_sig[n, :])

  return treated_sig

def bold_FC_pipeline(bold_sig, dt, fs):

  '''
  Takes np.array of BOLD signals with shape [node, step] and treats it for FC analysis.
  Pipeline based on Cabral et al. 2011 Neuroimage and Deco et al 2021 Science Advances

  Args:
    bold_sig (array): array with shape [time step, node] containing BOLD signals for all nodes
    dt (float): time step of original signal (ms)
  Returns:
    processed_sig (array): resampled bold signal
  '''

  ns = bold_sig.shape[-1]
  frs = 1/(0.72)
  fs = 1e3/dt
  nrs = int(fs/frs)
  bdr = int(60 * fs) # Time to remove at the beginning and end of signal, to avoid boundary effects from the Balloon-Windkessel model

  n_samples = int(frs * (bold_sig.shape[-1] - 2 * bdr) / fs)
  treated_sig = np.zeros((bold_sig.shape[0], n_samples))

  for n in range(bold_sig.shape[0]):
      treated_sig[n, :] = sig.resample(bold_sig[n, bdr:-bdr], n_samples)

  b, a = sig.butter(2, [0.008, 0.08], btype = 'band', output = 'ba', fs = frs)

  treated_sig = sig.filtfilt(b, a, treated_sig, axis = -1)

  return treated_sig

def plot_BOLD(BOLD, n = 10, fs = 0.5):

  '''
  Plots BOLD signal for first n nodes.

  Args:
    BOLD (array): processed BOLD signal with shape [node, sample]
    T_sig (float): duration of recording
    n (int): number of nodes to be shown
    fs (float): sampling frequency of signal
  '''

  plt.figure(figsize=(12, 6))
  plt.plot(np.arange(0, BOLD.shape[-1])/fs, BOLD[:n, :].T + np.r_[:n], color = 'k')
  for i in range(n):
    plt.axhline(y = i, color = 'k', alpha = 0.4)
  plt.yticks([i for i in range(n)], ['Node {}'.format(i) for i in range(n)], fontsize = 14)
  plt.xlabel('Time (s)', fontsize = 14)
  plt.ylim([-1, n])
  plt.title('BOLD Signal', fontsize = 14)
  plt.tight_layout()
  plt.show()

def plot_BOLD_standardized(BOLD, n = 10, fs = 0.5):

  '''
  Plots BOLD signal for first n nodes.

  Args:
    BOLD (array): processed BOLD signal with shape [node, sample]
    T_sig (float): duration of recording
    n (int): number of nodes to be shown
    fs (float): sampling frequency of signal
  '''

  BOLD_norm = BOLD.copy()
  for i in range(BOLD.shape[0]):
    BOLD_norm[i, :] = (BOLD[i, :] - np.mean(BOLD[i, :])) / np.std(BOLD[i, :])

  plt.figure(figsize=(12, 6))
  plt.plot(np.arange(0, BOLD_norm.shape[-1])/fs, BOLD_norm[:n, :].T / 3 + np.r_[:n], color = 'k')
  for i in range(n):
    plt.axhline(y = i, color = 'k', alpha = 0.4)
  plt.yticks([i for i in range(n)], ['Node {}'.format(i) for i in range(n)], fontsize = 14)
  plt.xlabel('Time (s)', fontsize = 14)
  plt.ylim([-1, n])
  plt.title('BOLD Signal - Standardized', fontsize = 14)
  plt.tight_layout()
  plt.show()

# Neural Mass Dynamics

## Activation Function

Neural mass dynamics are commonly modeled as systems of differential equations similar to the one below. $r$ represents the population firing rate, $x$ the external input and $\tau$ a constant that describes the timescale of changes in activity.

$$ \tau\frac{dr(t)}{dt} = -r(t) + F(x)  \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ (1)$$

F(I), often called **activation function**, is a function that represents how the neural population reacts to external inputs. Most neural mass models use sigmoid (or similar) activation functions. In this case, our sigmoid function has the form:

$$ F(x) = \frac{F_{max}}{1+e^{-\frac{x-\mu}{\sigma}}} \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ (2)$$

In this block, we will build an implementation of $F(x)$ and explore what we can change by manipulating its parameters.

**Exercise 1**

In [ ]:
def F(x, F_max, mu, sigma):
    """
    Population activation function.

    Args:
      x (float): external input
      F_max (float): the gain of the function
      mu (float): activation threshold
      sigma (float): the threshold of the function

    Returns:
      float: the population activation response F(x) for input x
    """
    #################################################
    # Fill out f = ... and comment the following line
    # Hint: you can use the numpy function np.exp() to compute an exponential function.
    #  NotImplementedError("Please complete coding exercise")
    #################################################

    f = F_max/(1+np.exp(-(x-mu)/sigma))

    return f

In [ ]:
# @markdown Code solution (click "show code" to copy)

# Copy the solution below

# f = F_max / (1 + np.exp(-(x - mu)/sigma))

In [ ]:
# @markdown Execute this cell to enable the widget!
# @markdown Play around with the sliders and explore the following questions:

# @markdown 1. What characteristics of the activation function can you vary by
# @markdown changing each parameter?

# @markdown 2. How do you think each parameter impacts population dynamics?


def F_exploration(F_max = 1, mu = 1, sigma = 0.25):

  pars = default_parameters()  # get default parameters
  x = np.arange(-1, 3, .01)      # set the range of input

  plt.figure(figsize=(15, 8))

  f = F(x, pars['F_max'], pars['mu'], pars['sigma'])
  plt.plot(x, f, linewidth = 4)
  f = F(x, F_max, pars['mu'], pars['sigma'])
  plt.plot(x, f, '--', alpha = 0.5, linewidth = 4)
  f = F(x, pars['F_max'], mu, pars['sigma'])
  plt.plot(x, f, '--', alpha = 0.5, linewidth = 4)
  f = F(x, pars['F_max'], pars['mu'], sigma)
  plt.plot(x, f, '--', alpha = 0.5, linewidth = 4)

  plt.legend(['Default', "$F_m$ = {}".format(F_max), '$\mu = ${}'.format(mu), '$\sigma = ${}'.format(sigma)], fontsize = 14)
  plt.xlabel('x (a.u.)', fontsize=14)
  plt.ylabel('F(x)', fontsize=14)
  plt.show()

_ = widgets.interact(F_exploration, F_max=(0.1, 2, 0.1), mu = (0.1, 2, 0.1), sigma = (0.05, 0.5, 0.05))

## Single Population Dynamics

In this block, we will implement single population dynamics by using the following description and the $F(x)$ function we defined before.

$$ \tau\frac{dr(t)}{dt} = -r(t) + F(I_{ext}(t))  \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ (3)$$

While this equation describes the neural dynamics in continuous time, we will have to discretize it in order to be able to solve it. For that, we need to approximate the solution of the equation using a numerical method, which works in discrete time. The simplest approach is the **Euler Method**, which can be defined as:

$$ r(t + dt) = r(t) + dt*dr(t)  \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ (4)$$

Where $dr(t)$ can be derived as follows:

$$ \tau\frac{r(t+dt) - r(t)}{dt} = -r(t) + F(I_{ext}(t)) \ \ \Leftrightarrow \ \ r(t+dt) = r(t) + dt * \frac{1}{\tau}[-r(t) + F(I_{ext}(t))] $$


In this block, we will build an implementation of the method described above where we can regulate simulation runtime **T** and the time step **dt**. Then, we will see how the changes in the parameters explored in the first exercise, and in the time constant **$\tau$**, impact neural dynamics.

**Exercise 2**

In [ ]:
def simulate_node(params, I_ext, T, dt):
    """
    Simulates activity for a single neural population.

    Args:
      params (dict): relevant parameters for simulation
      I_ext (array): external input
      T (float): total simulation time in ms
      dt (float): integration time step in ms

    Returns:
      r (array): population activity for all time steps
    """
    #################################################
    # Fill out dr = ... and comment the following line
    # Tip: Don't forget to specify all the required arguments inside the F() function
    raise NotImplementedError("Please complete coding exercise")
    #################################################

    # Sets up required parameters
    F_max = params['F_max']
    mu = params['mu']
    sigma = params['sigma']
    tau = params['tau_E']

    # Initializes activity array
    r = np.zeros(int(np.floor(T/dt)))

    for i in range(1, int(np.floor(T/dt))):
        dr = ...
        r[i] = r[i-1] + dt * dr

    return r

In [ ]:
# @markdown Code solution (click "show code" to copy)

# Copy the solution below

# dr = (1/tau) * (-r[i-1] + F(I_ext[i-1], F_max, mu, sigma))

In [ ]:
# @markdown Execute this cell to enable the widget!
# @markdown Play around with the sliders and explore the following questions:
# @markdown 1. What changes in dynamics are caused by manipulating the parameters of the activation function? Did you guess them correctly in the previous exercise?

def single_exploration(F_max = 1, mu = 1, sigma = 0.25):

  # Simulation parameters
  T = 175 #[ms] Simulation time
  dt = 0.2 #[ms] Integration time step
  t_vec = np.arange(0, T, dt) # Array with time steps

  # Externally applied current
  t_start = 50
  t_end = 70
  I_ext = pulse(0.5, 25, 50, T, dt) + pulse(2, 75, 100, T, dt) + pulse(-0.5, 125, 150, T, dt)

  # Simulation parameters
  plt.figure(figsize=(15, 8))
  plt.subplot(211)

  pars = default_parameters()
  r_E = simulate_node(pars, I_ext, T, dt)
  plt.plot(t_vec, r_E, linewidth = 4)

  pars = default_parameters(F_max = F_max)
  r_E = simulate_node(pars, I_ext, T, dt)
  plt.plot(t_vec, r_E, '--', alpha = 0.5, linewidth = 4)

  pars = default_parameters(mu = mu)
  r_E = simulate_node(pars, I_ext, T, dt)
  plt.plot(t_vec, r_E, '--', alpha = 0.5, linewidth = 4)

  pars = default_parameters(sigma = sigma)
  r_E = simulate_node(pars, I_ext, T, dt)
  plt.plot(t_vec, r_E, '--', alpha = 0.5, linewidth = 4)

  plt.axhline(y = 0, color = 'k', alpha = 0.4)
  plt.xticks([])
  plt.ylabel('Activity (a.u.)', fontsize = 14)
  #plt.ylim([-0.05, 1.05 * pars['F_max']])
  plt.legend(['Default', r'$F_m = ${}'.format(F_max), '$\mu = ${}'.format(mu), '$\sigma = ${}'.format(sigma)], fontsize = 14)

  plt.subplot(212)
  plt.plot(t_vec, I_ext, linewidth = 4)
  plt.xlabel('Time (ms)', fontsize = 14)
  plt.ylabel(r'$I_{ext}$ (a.u.)', fontsize = 14)
  plt.tight_layout()
  plt.show()

_ = widgets.interact(single_exploration, F_max=(0.1, 2, 0.1), mu = (0.1, 2, 0.1), sigma = (0.05, 1, 0.05))

In [ ]:
# @markdown Execute this cell to enable the widget!
# @markdown Play around with the slider and explore the following questions:

# @markdown 2. How does the time constant $\tau$ impact population activity?

def single_exploration(tau = 2.5):

  # Simulation parameters
  T = 175 #[ms] Simulation time
  dt = 0.2 #[ms] Integration time step
  t_vec = np.arange(0, T, dt) # Array with time steps

  # Externally applied current
  t_start = 50
  t_end = 70
  I_ext = pulse(1, 25, 50, T, dt) + pulse(2, 75, 100, T, dt) + pulse(-1, 125, 150, T, dt)

  # Simulation parameters
  plt.figure(figsize=(15, 8))
  plt.subplot(211)

  pars = default_parameters()
  r_E = simulate_node(pars, I_ext, T, dt)
  plt.plot(t_vec, r_E, linewidth = 4)

  pars = default_parameters(tau_E = tau)
  r_E = simulate_node(pars, I_ext, T, dt)
  plt.plot(t_vec, r_E, '--', alpha = 0.5, linewidth = 4)

  plt.axhline(y = 0, color = 'k', alpha = 0.4)
  plt.xticks([])
  plt.ylabel('Activity (a.u.)', fontsize = 14)
  plt.ylim([-0.05, 1.05 * pars['F_max']])
  plt.legend(['Default', r'$\tau = ${} ms'.format(tau)], fontsize = 14)

  plt.subplot(212)
  plt.plot(t_vec, I_ext, linewidth = 4)
  plt.xlabel('Time (ms)', fontsize = 14)
  plt.ylabel(r'$I_{ext}$ (a.u.)', fontsize = 14)
  plt.tight_layout()
  plt.show()

_ = widgets.interact(single_exploration, tau = (0.5, 10, 0.5))

## Excitatory and Inhibitory Populations - The Wilson-Cowan Model

Now that we established and explored how single neural masses work, we can go on to the next step. In their woriginal work, Wilson and Cowan conceptualized neural masses as coupled excitatory and inhibitory populations. The populations can then be described by the following equations:

$$$$
$$ \tau_E\frac{dr^E(t)}{dt} = -r^E(t) + F[c_{EE}r^E(t) - c_{EI}r^I(t) + I_{ext}(t)] \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ (5)$$

$$$$
$$ \tau_I\frac{dr^I(t)}{dt} = -r^I(t) + F[c_{IE}r^E(t) - c_{II}r^I(t)] \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ (6) $$
$$$$

where $c_{xy}$ represents the weight of the coupling from population $y$ to population $x$. Also, each population works in a different timescale, so it is necessary to define the individual time constants $\tau_E$ and $\tau_I$.

By coupling neural masses that have an opposite influence (excitatory/inhibitory) on each other, we are able to obtain more complex regimes of activity than the one in our simple neural mass. In this section, we will explore how such regimes are dependent on variables such as the external input ($I_{ext}$) and the time constants.

**Note:** From now on, we will consider $c_{II}$ to be = 0 and fix the other coupling weights to values found in literature. Feel free to explore and play around with the parameters after the tutorial.

**Exercise 3**

In [ ]:
def simulate_EI(pars, I_ext, T, dt):
    """
    Simulates activity from two populations of interacting excitatory and inhibitory populations.

    Args:
      params (dict): relevant parameters for simulation
      I_ext (array): external input
      T (float): total simulation time in ms
      dt (float): time step of simulation

    Returns:
      r_E (array): excitatory population activity for all time steps
      r_I (array): inhibitory population activity for all time steps
    """
    #################################################
    # We will implement the inputs to the F function described above
    # Fill out E_input = ... and I_input = ... and comment line below
    raise NotImplementedError("Please complete coding exercise")
    #################################################

    # Sets up required parameters
    F_max = pars['F_max']
    mu = pars['mu']
    sigma = pars['sigma']
    tau_E = pars['tau_E']
    tau_I = pars['tau_I']
    cEE = pars['cEE']
    cEI = pars['cEI']
    cIE = pars['cIE']

    # Initializes activity array
    r_E = np.zeros(int(np.floor(T/dt)))
    r_I = np.zeros(int(np.floor(T/dt)))

    for i in range(1, int(np.floor(T/dt))):
        E_input = ...
        I_input = ...

        drE = (1/tau_E) * (-r_E[i-1] + F(E_input, F_max, mu, sigma))
        drI = (1/tau_I) * (-r_I[i-1] + F(I_input, F_max, mu, sigma))

        r_E[i] = r_E[i-1] + dt * drE
        r_I[i] = r_I[i-1] + dt * drI

    return r_E, r_I

In [ ]:
# @markdown Code solution (click "show code" to copy)

# Copy the solution below

# E_input = cEE * r_E[i-1] - cEI * r_I[i-1] + I_ext[i-1]
# I_input = cIE * r_E[i-1]

In [ ]:
# @markdown Execute this cell to enable the widget!
# @markdown Play around with the slider and explore the following questions:

# @markdown 1. How many different activity regimes can we achieve by changing the external input? Where are the points of transition?

# @markdown 2. With $I_{amp}$ = 0.4, increase **$\tau_E$**.
# @markdown What happens to the dynamics?

# @markdown **Note**: in this case, we fix $\tau_I$ to be $2 * \tau_E$. Feel free to explore other combinations after the tutorial.



def interactive_plot_EI(I_amp = 0.1, tau_E = 2.5):
  """
  Population activation function.

  Expecxts:
    I_amp : the amplitude of external input

  Returns:
    plot the F-I curve with give parameters
  """
  T = 2000
  dt = 0.2
  t_vec = np.arange(0, T, dt)
  pars = default_parameters(sd_noise = 0, tau_E = tau_E, tau_I = 2 * tau_E)

  I_ext = I_amp * np.ones(len(t_vec))

  r_E, r_I = simulate_EI(pars, I_ext, T, dt)

  plt.figure(figsize=(9, 4))
  plt.plot(t_vec, r_E, color = 'darkred', linewidth = 3)
  plt.plot(t_vec, r_I, color = 'steelblue', linewidth = 3)
  plt.axhline(y = 0, color = 'k', alpha = 0.4)
  plt.ylabel('Activity (a.u.)', fontsize = 14)
  plt.ylim([-0.05, 1.05 * pars['F_max']])
  plt.legend(['E', 'I'], fontsize = 14)
  plt.xlabel('Time (ms)', fontsize = 14)
  plt.title(r'$\tau_E = ${} ms // $\tau_I = ${} ms'.format(tau_E, 2 * tau_E), fontsize = 14)
  plt.tight_layout()
  plt.show()


_ = widgets.interact(interactive_plot_EI, I_amp=(0, 1, 0.01), tau_E = (2.5, 50, 2.5))

We will now add a noise term to our equations, representic a level of stochasticity observed in real neural dynamics. We can do that by adding a term to our equations that will represent additive Gaussian noise ($\xi$) with a chosen standard deviation:

$$$$
$$ \tau_E\frac{dr^E(t)}{dt} = -r^E(t) + F[c_{EE}r^E(t) - c_{EI}r^I(t) + I_{ext}(t)+ \xi(t)] \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ (7) $$

$$$$
$$ \tau_I\frac{dr^I(t)}{dt} = -r^I(t) + F[c_{II}r^E(t) + \xi(t)]  \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ (8)$$
$$$$

In this block, we will explore how noise impacts dynamics.

**Exercise 4**

In [ ]:
# @markdown Run this cell to update the simulator with noise

def simulate_EI(pars, I_ext, T, dt):
    """
    Simulates activity from two populations of interacting excitatory and inhibitory populations.

    Args:
      params (dict): relevant parameters for simulation
      I_ext (array): external input
      T (float): total simulation time in ms
      dt (float): time step of simulation

    Returns:
      r_E (array): excitatory population activity for all time steps
      r_I (array): inhibitory population activity for all time steps

    """

    # Sets up required parameters
    F_max = pars['F_max']
    mu = pars['mu']
    sigma = pars['sigma']
    tau_E = pars['tau_E']
    tau_I = pars['tau_I']
    cEE = pars['cEE']
    cEI = pars['cEI']
    cIE = pars['cIE']
    sd_noise = pars['sd_noise']

    # Initializes activity array
    r_E = np.zeros(int(np.floor(T/dt)))
    r_I = np.zeros(int(np.floor(T/dt)))

    for i in range(1, int(np.floor(T/dt))):
        E_input = cEE * r_E[i-1] - cEI * r_I[i-1] + I_ext[i-1] + sd_noise * np.random.randn()
        I_input = cIE * r_E[i-1] + sd_noise * np.random.randn()

        dRE = (1/tau_E) * (-r_E[i-1] + F(E_input, F_max, mu, sigma))
        dRI = (1/tau_I) * (-r_I[i-1] + F(I_input, F_max, mu, sigma))

        r_E[i] = r_E[i-1] + dt * dRE
        r_I[i] = r_I[i-1] + dt * dRI

    return r_E, r_I

In [ ]:
# @markdown Execute this cell to enable the widget!
# @markdown We will now study a node that is close to the point of transition
# @markdown between the **low activity** and **oscillatory** regimes ($I_{ext} = 0.30$). Play around with the slider and explore the following questions:

# @markdown 1. What happens when you manipulate the amount of noise in the system?

def interactive_plot_EI(sd_noise = 0):
  """
  Population activation function.

  Expecxts:
    I_amp : the amplitude of external input

  Returns:
    plot the F-I curve with give parameters
  """
  T = 2000
  dt = 0.2
  t_vec = np.arange(0, T, dt)
  pars = default_parameters(sd_noise = sd_noise)

  I_ext = 0.3 * np.ones(len(t_vec))

  r_E, r_I = simulate_EI(pars, I_ext, T, dt)

  plt.figure(figsize=(9, 4))
  plt.plot(t_vec, r_E, color = 'darkred', linewidth = 3)
  plt.plot(t_vec, r_I, color = 'steelblue', linewidth = 3)
  plt.axhline(y = 0, color = 'k', alpha = 0.4)
  plt.ylabel('Activity (a.u.)', fontsize = 14)
  plt.ylim([-0.05, 1.05 * pars['F_max']])
  plt.legend(['E', 'I'], fontsize = 14)
  plt.xlabel('Time (ms)', fontsize = 14)
  plt.title(r'Noise sd = {}'.format(sd_noise), fontsize = 14)
  plt.tight_layout()
  plt.show()


_ = widgets.interact(interactive_plot_EI, sd_noise=(0, 0.2, 0.01))

# Large Scale Models of Coupled Neural Masses

## Coupling Two Wilson-Cowan Nodes

When performing simulations of large-scale neural activity, it is common to use networks of coupled neural masses. For example, local dynamics of discretized regions in the brain can be described the Wilson-Cowan model and connections between distant regions in the brain can be represented using a connectivity matrix $W_{ij}$. Now, $r^E$ and $r^I$ are no longer single values, but vectors representing the activities of each local population.

We can now make use of the equations above and add an extra term that represents the connections between distant neural masses.

$$$$
$$ \tau_E\frac{dr_i^E(t)}{dt} = -r_i^E(t) + F[c_{EE}r_i^E(t) - c_{EI}r_i^I(t) + C\sum_j W_{ij} r_j^E(t) + P + I_{ext}(t) + \xi(t)] \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ (9)$$

$$$$
$$ \tau_I\frac{dr_i^I(t)}{dt} = -r_i^I(t) + F[c_{IE}r_i^E(t) - c_{II}r_i^I(t) + \xi(t)]  \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ (10) $$
$$$$

where we added the term $W_{ij} r_j^E(t)$ to the $r^E$ dynamics. Note that long-range connections seem to only involve excitatory masses. This is a common approach in large-scale models of the human brain, since long-range nerve projections are almost exclusively excitatory.

We also added the term $P$, similarly to previous studies (https://doi.org/10.1371/journal.pcbi.1006007). This term controls the intrinsic excitability of the excitatory populations and it is common to define in such a way that an uncoupled neural mass, with no external input, is in the verge of the state transition.

$$$$

For this first approach, we will take a simple network of reciprocally coupled nodes. Therefore, the connectivity matrix is simply:

$$ W_{ij} = \begin{bmatrix}    0 & 1 \\ 1 & 0 \\ \end{bmatrix}$$

In this exercise, we will vary $C$ to increase or decrease reciprocal coupling between masses and observe its effects on the behavior of our simple network.

**Exercise 5**

In [ ]:
def simulate_network(pars, I_ext, C, W, T, dt):
    """
    Simulates activity from a network of Wilson-Cowan nodes, connected according to W.

    Args:
      params (dict): relevant parameters for simulation
      I_ext (array): external input
      W (array): connectivity matrix
      T (float): total simulation time in ms
      dt (float): integration time step in ms

    Returns:
      r_E (array): excitatory population activity for all time steps
      r_I (array): inhibitory population activity for all time steps
    """

    #################################################
    # Fill out the ... in E_input with an implementation of long range connectivity and comment line below
    # Hint: you can use the operator @ in python to perform matrix multiplication
    raise NotImplementedError("Please complete coding exercise")
    #################################################

    # Sets up required parameters
    F_max = pars['F_max']
    mu = pars['mu']
    sigma = pars['sigma']
    tau_E = pars['tau_E']
    tau_I = pars['tau_I']
    cEE = pars['cEE']
    cEI = pars['cEI']
    cIE = pars['cIE']
    sd_noise = pars['sd_noise']
    P = pars['P']

    n_nodes = W.shape[0]

    # Initializes activity array
    r_E = np.zeros((n_nodes, int(np.floor(T/dt))))
    r_I = np.zeros((n_nodes, int(np.floor(T/dt))))

    for i in range(1, int(np.floor(T/dt))):
        E_input = cEE * r_E[:, i-1] - cEI * r_I[:, i-1] + ... + I_ext[:, i-1] + P + sd_noise * np.random.randn(n_nodes)
        I_input = cIE * r_E[:, i-1] + sd_noise * np.random.randn(n_nodes)

        dRE = (1/tau_E) * (-r_E[:, i-1] + F(E_input, F_max, mu, sigma))
        dRI = (1/tau_I) * (-r_I[:, i-1] + F(I_input, F_max, mu, sigma))

        r_E[:, i] = r_E[:, i-1] + dt * dRE
        r_I[:, i] = r_I[:, i-1] + dt * dRI

    return r_E, r_I

In [ ]:
# @markdown Code solution (click "show code" to copy)

# Copy the solution below

# E_input = E_input = cEE * r_E[:, i-1] - cEI * r_I[:, i-1] + C*W@r_E[:, i-1] + I_ext[:, i-1] + P + sd_noise * np.random.randn(n_nodes)

In [ ]:
# @markdown Execute this cell to enable the widget!
# @markdown Play around with the slider and explore the following questions:

# @markdown 1. How does the coupling between nodes influence the similarity between
# @markdown the activity of their excitatory populations?


def interactive_plot_2n(C = 0):
  """
  Args:
    I_amp : the amplitude of external input

  Returns:
    plot the F-I curve with give parameters
  """
  # Simulation parameters
  T = 2100 #[ms] Simulation time
  dt = 0.2 #[ms] Integration time step
  t_vec = np.arange(0, T-100, dt) # Array with time steps

  # Connectivity matrix
  W = np.array([[0, 1], [1, 0]])

  # Simulation parameters
  pars = default_parameters(P = 0.30, sd_noise = 0.1)
  I_ext = np.zeros((W.shape[0], int(np.floor(T/dt))))

  r_E, r_I = simulate_network(pars, I_ext, C, W, T, dt)

  plt.figure(figsize=(12, 6))
  plt.plot(t_vec, r_E[:, 500:].T + np.r_[:W.shape[0]], color = 'darkred')
  plt.axhline(y = 0, color = 'k', alpha = 0.4)
  plt.axhline(y = 1, color = 'k', alpha = 0.4)
  plt.axhline(y = 2, color = 'k', alpha = 0.4)
  plt.yticks([0.5, 1.5], ['Node 0', 'Node 1'], fontsize = 14)
  plt.xlabel('Time (ms)', fontsize = 14)
  plt.text(0, W.shape[0]+0.1, 'Correlation = {}'.format(np.around(np.corrcoef(r_E[0, 500:], r_E[1, 500:])[0,1], 3)), fontsize = 14)
  plt.ylim([0, 1.15 * W.shape[0]])
  plt.tight_layout()
  plt.show()

_ = widgets.interact(interactive_plot_2n, C=(0, 1, 0.02))

## Connectomics and Human Brain Modeling


The neural network of the human cortex has patterns of connectivity that are significantly more complex than the simple network we just modelled. Therefore, when modeling human brain dynamics arise, two important problems arise:
1. How to discretize the human cortex into regions that we can model using neural masses?
2. How to accurately model the structural connectivity between said regions?

Thankfully, imaging techniques such as magnetic resonance imaging (MRI) allow us to discretize the human brain into anatomically or functionally specific areas and approximate the strenght of the connections between them, giving us our connectivity matrix.

In this section, we will load a connectivity matrix $W_{ij}$ representing the average structural connectivity between 78 regions of the cortex of 32 healthy sujects, using the AAL atlas. Then, we will explore how changing the global coupling $C$ impacts on neural dynamics.

In [ ]:
# @markdown Execute this cell to load and visualize the human structural connectivity matrix!

# @markdown **Note:** in the brain visualization, only the 5% strongest weights are shown, for simplicity of visualization


W, coords, labels = load_connectivity()


plt.figure(figsize = (15, 15))
im = plt.imshow(W, cmap = 'hot')
c = plt.colorbar(im, fraction=0.046, pad=0.04)
c.set_label('Weigth', fontsize = 14)
plt.xticks([])
plt.yticks(np.arange(W.shape[0]), labels)
plt.clim([0, 1])
plt.show()

density = 0.1
th = np.sort(W.flatten())[int(np.size(W)*(1-density))]

fig = plt.figure(figsize = (15, 8))
ax1 = plt.subplot(121)
ax = plotting.plot_connectome(W,coords,display_mode='z', edge_threshold=th, edge_vmin=-1, edge_vmax=1, figure=fig, node_size=100, node_color='k', alpha = 0.5, axes = ax1)
ax2 = plt.subplot(122)
ax = plotting.plot_connectome(W,coords,display_mode='x', edge_threshold=th, edge_vmin=-1, edge_vmax=1, figure=fig, node_size=100, node_color='k', alpha = 0.5, axes = ax2)

plt.show()

**Exercise 6**

In [ ]:
# @markdown Execute this cell to enable the widget!
# @markdown Play around with the slider and explore the following questions:

# @markdown 1. How does the global coupling (C) between nodes influence the average correlation between node activity?

# @markdown 2. What happens when we increase C above a certain value?

# @markdown **Note:** only the first 5 nodes are shown and considered when calculating the correlation

def interactive_plot_brain(W, C = 0):
  """
  Args:
    I_amp : the amplitude of external input

  Returns:
    plot the F-I curve with give parameters
  """
  # Simulation parameters
  T = 5100 #[ms] Simulation time
  dt = 0.5 #[ms] Integration time step
  t_vec = np.arange(0, T-100, dt) # Array with time steps

  # Simulation parameters
  pars = default_parameters(P = 0.31, sd_noise = 0.05)
  I_ext = np.zeros((W.shape[0], int(np.floor(T/dt))))

  r_E, r_I = simulate_network(pars, I_ext, C, W, T, dt)

  corr_mat = np.triu(np.corrcoef(r_E[:5, 200:]), k = 1)
  avg_corr = np.mean(corr_mat)

  plt.figure(figsize=(12, 6))
  plt.plot(t_vec, r_E[:5, 200:].T + np.r_[:5], color = 'darkred')
  for i in range(6):
    plt.axhline(y = i, color = 'k', alpha = 0.4)
  plt.yticks([i+0.5 for i in range(5)], ['Node {}'.format(i) for i in range(5)], fontsize = 14)
  plt.xlabel('Time (ms)', fontsize = 14)
  plt.text(0, 5.3, 'Average corr. = {}'.format(np.around(avg_corr, 3)), fontsize = 14)
  plt.ylim([0, 6])
  plt.tight_layout()
  plt.show()

_ = widgets.interact(interactive_plot_brain, W = widgets.fixed(W), C=(0, 0.5, 0.01))

## BOLD Functional Connectivity

Functional connectivity, as opposed to structural connectivity, represents functional coupling between different areas in the brain. This is often evaluated by correlations in activity which means that regions that are have no structural connections can be functionally connected.

Functional connectivity is often measured in the brain by measuring the correlation between fMRI derived **blood-oxygenation-level dependent (BOLD)** signals. BOLD signals represent haemodynamic responses, tied to increased metabolism from neurons in measured areas, which is used as a proxy for increased neural activity.

In our models, so far, we have been simulating population firing rates. The **Balloon-Windkessel model**, first developed by Buxton et al. (doi: 10.1002/mrm.1910390602) and then updated by Friston et al. (doi:10.1006/nimg.2000.0630), allows us to go from firing-rate activity to simulated BOLD signals. In short, activity from neural masses is run through a hemodynamic model specifying the coupling between activity and blood perfusion, which then underlies simulated BOLD signals.

**Exercise 7**

In [ ]:
# @markdown Run this cell to update the simulation function

def simulate_network(pars, I_ext, C, W, T, dt):
    """
    Population activation function.

    Args:
      params (dict): relevant parameters for simulation
      I_ext (array): external input
      W (array): connectivity matrix
      T (float): total simulation time in ms
      dt (float): integration time step in ms

    Returns:
      r_E (array): population activity for all time steps
    """

    # Sets up required parameters
    F_max = pars['F_max']
    mu = pars['mu']
    sigma = pars['sigma']
    tau_E = pars['tau_E']
    tau_I = pars['tau_I']
    cEE = pars['cEE']
    cEI = pars['cEI']
    cIE = pars['cIE']
    sd_noise = pars['sd_noise']
    P = pars['P']

    n_nodes = W.shape[0]

    # Initializes activity array
    r_E = np.zeros((n_nodes, int(np.floor(T/dt))))
    r_I = np.zeros((n_nodes, int(np.floor(T/dt))))

    print('Simulation running...')
    for i in tqdm(range(1, int(np.floor(T/dt)))):
        E_input = cEE * r_E[:, i-1] - cEI * r_I[:, i-1] + C*W @ r_E[:, i-1] + P + sd_noise * np.random.randn(n_nodes)
        I_input = cIE * r_E[:, i-1] + sd_noise * np.random.randn(n_nodes)

        dRE = (1/tau_E) * (-r_E[:, i-1] + F(E_input, F_max, mu, sigma))
        dRI = (1/tau_I) * (-r_I[:, i-1] + F(I_input, F_max, mu, sigma))

        r_E[:, i] = r_E[:, i-1] + dt * dRE
        r_I[:, i] = r_I[:, i-1] + dt * dRI

    return r_E, r_I

In [ ]:
# @markdown Run this cell to run a simulation of 6 minutes of activity. 5 minutes
# @markdown will be used to calculate the respective BOLD signal. The simulation and
# @markdown calculation of BOLD signals might take a while, so be patient!

# @markdown When the simulation finishes, try to answer the following questions:
# @markdown 1. How fast do the BOLD signals fluctuate, respective to previously observed population firing rates?
# @markdown 2. How uniform is the amplitude of the simulated BOLD signal across areas? Can you find an explanation for what you observe?

# Simulation parameters

T = 6 * 60 * 1000 # [ms] Simulation time
dt = 0.2 #[ms] Integration time step
t_vec = np.arange(0, T, dt) # Array with time steps

# Connectivity matrix
C = 0.13

# Simulation parameters
pars = default_parameters(P = 0.31, sd_noise = 0.01)
I_ext = 0

r_E, r_I = simulate_network(pars, I_ext, C, W, T, dt)

BOLD = balloon(r_E, dt)
BOLD_rs = bold_rs_pipeline(BOLD, dt, fs = 1e3/dt)
BOLD = bold_FC_pipeline(BOLD, dt, fs = 1e3/dt)
BOLD /= np.max(np.abs(BOLD)) # Normalize for visualization
BOLD_rs /= np.max(np.abs(BOLD_rs)) # Normalize for visualization

plot_BOLD(BOLD_rs, fs = 1/0.72, n = 15)
plot_BOLD_standardized(BOLD_rs, fs = 1/0.72, n = 15)

Now that we have established how to simulate BOLD signals from the human cortex, it’s time to look at functional connectivity. We will use the pair-wise Pearson’s Correlation Coefficient as our estimate of functional connectivity, which means that $FC_{ij}$ can be defined as:

$$$$

$$
FC_{ij} = corrcoef(BOLD_i, BOLD_j)
$$

$$$$

In the following exercise, you will implement a function to calculate the functional connectivity matrix of our model, given its simulated BOLD signal.


**Exercise 8**

In [ ]:
def compute_FC(BOLD):

    #################################################
    # Fill out FC_mat[i, j] = ... and comment line below
    # Hint: use the np.corrcoef() function to calculate the correlation between two arrays.
    # np.corrcoef() outputs a matrix [[corr(i,i), corr(i,j)], [corr(j,i), corr(j,j)]] so make sure to select the right element!
    raise NotImplementedError("Please complete coding exercise")
    #################################################

    n_nodes = np.shape(BOLD)[0]
    FC_mat = np.zeros((n_nodes, n_nodes))

    #Fills in upper triangle of matrix, because it's symmetrical
    for i in range(n_nodes):
        for j in range(i+1, n_nodes):
            FC_mat[i, j] = ...

    # Fills in rest of matrix. Correlation between node i and node j is the same as node j and node i
    FC_mat += FC_mat.T

    return FC_mat

In [ ]:
# @markdown Code solution (click "show code" to copy)

# Copy the solution below

# FC_mat[i, j] = np.corrcoef(BOLD[i, :], BOLD[j, :])[0, 1]

# While the code was written in this way for you to better conceptualize the pairwise
# correlation between areas that entails FC, there is actually a much simpler and
# more efficient way to calculate it:

# FC_mat = np.corrcoef(BOLD)

# When presented with a multidimensional array, the numpy function corrcoef
# computes the pairwise correlation between the rows of the matrix, by default.

In [ ]:
#@markdown Run this cell to visualize FC and compare it with SC. Try to answer the following question:
#@markdown 1. Do you see the same structure organization that you previously observed for FC?
#@markdown 2. How do the structural and functional connectivities compare? Can you identify strong functional links between weakly connected regions and vice-versa?


FC_mat = compute_FC(BOLD)

plt.figure(figsize = (15, 15))
im = plt.imshow(FC_mat, cmap = 'viridis')
c = plt.colorbar(im, fraction=0.046, pad=0.04)
plt.clim([0, 1])
c.set_label('Functional Connectivity', fontsize = 14)
plt.xticks([])
plt.yticks(np.arange(W.shape[0]), labels)
plt.show()


plt.figure(figsize = (18, 6))
plt.subplot(131)
plt.title('FC Matrix')
im = plt.imshow(FC_mat, cmap = 'viridis')
plt.clim([0, 1])
plt.xticks([])
plt.yticks([])
plt.subplot(132)
plt.title('SC Matrix')
im = plt.imshow(W, cmap = 'hot')
plt.clim([0, 1])
plt.xticks([])
plt.yticks([])
plt.subplot(133)
plt.title(r'Correlation FC vs. SC || $\rho$ = {}'.format(np.around(np.corrcoef(W.flatten()[:, ], FC_mat.flatten())[0, 1], 3)))
reg = LinearRegression().fit(W.flatten()[:, None], FC_mat.flatten()[:, None])
plt.plot(np.arange(0, 1, 0.01), reg.intercept_ + reg.coef_[0] * np.arange(0, 1, 0.01), 'r--')
plt.scatter(W.flatten()[:, ], FC_mat.flatten())
plt.xlabel('SC')
plt.ylabel('FC')
plt.legend(['Linear Reg. Fit'])
plt.tight_layout()
plt.show()

## Applications of Large-Scale Modeling: Functional Connectivity in Lesioned Networks

In this section, we will explore how lesions in the structural connectivity (e.g. in the case of stroke) might impact functional connectivity. Here, a lesion in a certain node will be emulated by removing all the connections too and from said node.

In this section, you will implement a lesion function and then select one (or more) node to lesion in your network. Then, simulations will be rerun to compare FC before and after lesion.

**Exercise 9**

In [ ]:
def lesion(SC, nodes):

  #################################################
  # Fill out ... and comment line below
  # Hint: Don't forget that you should lesion connections both to and from the specified nodes
  raise NotImplementedError("Please complete coding exercise")
  #################################################

  new_SC = SC.copy()
  for node in nodes:

    # Write your code here
    ...

  return new_SC

In [ ]:
# @markdown Code solution (click "show code" to copy)

# Copy the solution below

# new_SC[node, :] = 0
# new_SC[:, node] = 0

In [ ]:
# @markdown Run this cell to choose and visualize lesioned nodes.

# @markdown **Note:** only connections with weight larger than 0.3 are shown, for simplicity of visualization.

lesion_nodes = input('Write lesioned node(s), separated by commas (e.g. 0,1,2,3) and press Enter: ')

if isinstance(lesion_nodes[0], str):
  lesion_nodes = [int(node) for node in lesion_nodes.split(',')]

print('')
print('')
print('Lesioned nodes: ')
print('')
for node in lesion_nodes:
  print('Node {}: {}'.format(node, labels[node]))

fig = plt.figure(figsize = (15, 8))

W_les_test = W.copy()

for node in lesion_nodes:
  W_les_test[node, :] = 0
  W_les_test[:, node] = 0

density = 0.1
th = np.sort(W.flatten())[int(np.size(W)*(1-density))]

ax1 = plt.subplot(121)
ax = plotting.plot_connectome(W_les_test,coords,display_mode='z', edge_threshold = th, edge_vmin=-1, edge_vmax=1, figure=fig, node_size=100, node_color='k', alpha = 0.5, axes = ax1)
for node in lesion_nodes:
  plt.scatter(coords[node, 0], coords[node, 1], color = 'skyblue', s = 600)
ax2 = plt.subplot(122)
ax = plotting.plot_connectome(W_les_test,coords,display_mode='x', edge_threshold= th, edge_vmin=-1, edge_vmax=1, figure=fig, node_size=100, node_color='k', alpha = 0.5, axes = ax2)
for node in lesion_nodes:
  plt.scatter(coords[node, 1], coords[node, 2], color = 'skyblue', s = 600)


plt.show()

In [ ]:
# @markdown Execute this cell to run a simulation of 6 minutes of activity,
# @markdown compute BOLD signals and use 5 minutes of signal to obtain the FC of our lesioned network.
# @markdown This should take about 5 minutes, so be patient!

lesion_W = lesion(W, lesion_nodes)

lesion_r_E, lesion_r_I = simulate_network(pars, I_ext, C, lesion_W, T, dt)

lesion_signal = lesion_r_E

lesion_BOLD = balloon(lesion_signal, dt)
lesion_BOLD = bold_FC_pipeline(lesion_BOLD, dt, fs = 1e3/dt)
lesion_BOLD /= np.max(np.abs(lesion_BOLD)) # Normalize for visualization

lesion_FC_mat = compute_FC(lesion_BOLD)

for node in lesion_nodes:
  lesion_FC_mat[node, :] = 0
  lesion_FC_mat[:, node] = 0

In [ ]:
#@markdown Run this cell to visualize pre-lesion FC, post-lesion FC and their difference. Try to answer the following questions:
#@markdown 1. Can you identify clear differences in FC after lesion? Does it mainly increase or decrease?
#@markdown 2. Are the effects of the lesion contained in its vicinity or do they spread across the cortex?

plt.figure(figsize = (18, 6))
plt.subplot(131)
plt.title('Healthy FC Matrix')
im = plt.imshow(FC_mat, cmap = 'viridis')
plt.axhline(y = 38.5, color = 'w')
plt.axvline(x = 38.5, color = 'w')
plt.clim([0, 1])
plt.xticks([])
plt.yticks([])
plt.subplot(132)
plt.title('Lesioned FC Matrix')
im = plt.imshow(lesion_FC_mat, cmap = 'viridis')
plt.axhline(y = 38.5, color = 'w')
plt.axvline(x = 38.5, color = 'w')
for node in lesion_nodes:
  plt.axhline(y = node, color = 'k', linewidth = 6, alpha = 1)
  plt.axvline(x = node, color = 'k', linewidth = 6, alpha = 1)

plt.clim([0, 1])
plt.xticks([])
plt.yticks([])
plt.subplot(133)
plt.title('FC Difference')
im = plt.imshow(lesion_FC_mat - FC_mat, cmap = 'coolwarm')
plt.axhline(y = 38.5, color = 'w')
plt.axvline(x = 38.5, color = 'w')
for node in lesion_nodes:
  plt.axhline(y = node, color = 'k', linewidth = 6, alpha = 1)
  plt.axvline(x = node, color = 'k', linewidth = 6, alpha = 1)
plt.clim([-1, 1])
plt.xticks([])
plt.yticks([])
plt.tight_layout()
plt.show()

## Bonus: Store Data for Visualization in BrainX3
#### **Note:** This section applies to the Large-Scale Modeling tutorial at the DCBT2022 summer school. If you are following this tutorial at any other context, fell free to skip it.


You might have noticed that it is hard to conceptualize functional connectivity when just looking at a matrix, which does not allow us to put the results in the context of the brain. This is where platforms such as **BrainX3** come into hand.

In this section, we will learn how to extract our functional connectivity data to a format that can be loaded and visualized in BrainX3.

In [ ]:
# @markdown For now, I have written the code for you, so that BrainX3 can read your data properly.
# @markdown Run this cell to export 3 matrices representing FC pre and post-lesion and their difference

diff_mat = lesion_FC_mat - FC_mat

for node in lesion_nodes:
  diff_mat[node, :] = 0
  diff_mat[:, node] = 0
  lesion_FC_mat[node, :] = 0
  lesion_FC_mat[:, node] = 0


# This doesn't affect the actual data to be visualized in Brainx3
# It is here to ensure that the colormap is symmetric in BrainX3
diff_mat[0, 0] = np.max(np.abs(diff_mat))
diff_mat[1, 1] = -np.max(np.abs(diff_mat))

G_pre_lesion = nx.convert_matrix.from_numpy_array(FC_mat)
G_pos_lesion = nx.convert_matrix.from_numpy_array(lesion_FC_mat)
G_diff = nx.convert_matrix.from_numpy_array(diff_mat)

nx.set_node_attributes(G_pre_lesion, {key:value for (key,value) in zip(np.arange(78), coords[:, 0])}, "x")
nx.set_node_attributes(G_pre_lesion, {key:value for (key,value) in zip(np.arange(78), coords[:, 1])}, "y")
nx.set_node_attributes(G_pre_lesion, {key:value for (key,value) in zip(np.arange(78), coords[:, 2])}, "z")

nx.set_node_attributes(G_pos_lesion, {key:value for (key,value) in zip(np.arange(78), coords[:, 0])}, "x")
nx.set_node_attributes(G_pos_lesion, {key:value for (key,value) in zip(np.arange(78), coords[:, 1])}, "y")
nx.set_node_attributes(G_pos_lesion, {key:value for (key,value) in zip(np.arange(78), coords[:, 2])}, "z")

nx.set_node_attributes(G_diff, {key:value for (key,value) in zip(np.arange(78), coords[:, 0])}, "x")
nx.set_node_attributes(G_diff, {key:value for (key,value) in zip(np.arange(78), coords[:, 1])}, "y")
nx.set_node_attributes(G_diff, {key:value for (key,value) in zip(np.arange(78), coords[:, 2])}, "z")

values = dict()
for i in range(78):
  if i in lesion_nodes:
    values[i] = 1
  else:
    values[i] = 0

nx.set_node_attributes(G_pos_lesion, {key:value for (key, value) in zip(np.arange(78), values)}, "value")
nx.set_node_attributes(G_diff, {key:value for (key, value) in zip(np.arange(78), values)}, "value")

nx.write_gpickle(G_pre_lesion, "FC_pre_lesion.gpickle")
nx.write_gpickle(G_pos_lesion, "FC_post_lesion.gpickle")
nx.write_gpickle(G_diff, "FC_difference.gpickle")

files.download("FC_pre_lesion.gpickle")
files.download("FC_post_lesion.gpickle")
files.download("FC_difference.gpickle")

In [ ]:
# @title Functions

# Auxiliary Functions

def load_connectivity():

  ## Connectivity matrix ##

  # Define the remote file to retrieve
  remote_url = 'https://drive.google.com/uc?id=1tkwQbpl0-PYwasci1Z_ClRWwe7ZlM1vz'
  # Define the local filename to save data
  local_file = 'SC_AAL78.npy'
  # Make http request for remote file data
  gdown.download(remote_url, local_file, quiet = True)

  # Gets connectivity information
  connect = np.load('SC_AAL78.npy')

  ## Coordinates ##

  # Define the remote file to retrieve
  remote_url = 'https://drive.google.com/uc?id=1U-sUjSEt2e3cCz6VWnNbccDVv_aBYRkJ'
  # Define the local filename to save data
  local_file = 'AAL78_coords.npy'
  # Make http request for remote file data
  gdown.download(remote_url, local_file, quiet = True)

  # Gets coordinate information
  coords = np.load('AAL78_coords.npy')

  ## Connectome labels ##

  # Define the remote file to retrieve
  remote_url = 'https://drive.google.com/uc?id=1E_s6tNXlKSfkD2gTD4cQAhSxl6LjM5nj'
  # Define the local filename to save data
  local_file = 'labels.npy'
  # Make http request for remote file data
  gdown.download(remote_url, local_file, quiet = True)

  # Gets coordinate information
  labels = np.load('labels.npy')

  # This sorts connectivity so that nodes are grouped by hemisphere
  order = np.concatenate((np.arange(0, 78, 2), np.arange(1, 78, 2)))

  connect = np.array([[connect[i][j] for j in order] for i in order])
  coords = np.array([coords[i] for i in order])
  labels = np.array([labels[i] for i in order])

  return connect, coords, labels

def default_parameters(**kwargs):
  pars = {}

  # Activation function parameters
  pars['F_max'] = 1 # maximum activation
  pars['mu'] = 1 # activation threshold
  pars['sigma'] = 0.25 # activation function slope

  # Time constants for both populations
  pars['tau_E'] = 2.5 # (ms) Excitatory time constant (tuned to gamma oscillations)
  pars['tau_I'] = 5 # (ms) Inhibitory time constant

  # Standard deviation of additive noise
  pars['sd_noise'] = 0.01

  # Couplings from excitatory to inhibitory population
  pars['cEE'] = 3.5 # E->E coupling
  pars['cIE'] = 3.75 # E->I coupling
  pars['cEI'] = 2.5 # I->E coupling

  # Parameter to control excitability of excitatory population
  pars['P'] = 0.31 # Defined to put the node at the transition between stable and oscillatory activity

  # External parameters if any
  pars.update(kwargs)

  return pars

def pulse(I, t_start, t_end, T, dt):

  I_vec = np.zeros(int(np.floor(T/dt)))
  I_vec[int(np.floor(t_start/dt)):int(np.floor(t_end/dt))] = I

  return I_vec

def balloon(signals, dt):

  '''
  Implementation of Balloon-Windkessel model for hemodynamics,
  as implemented by Deco et al. 2009 PNAS. Takes firing-rate activity from
  model and transforms it into a simulated BOLD fMRI signal

  Args:
    signals (array): array with shape [node, sample] containing model activity
    dt (float): time step of original signal (ms)

  Returns:
    BOLD (array): simulated BOLD signal for all nodes [node, sample]
  '''

  print('Applying Balloon-Windkessel Model...')

  BOLD = np.zeros(np.shape(signals))

  k = 0.65 # [1/s] rate of signal decay
  gamma = 0.41  # [1/s] rate of flow-dependent elimination
  tau = 0.98 # [s] hemodynamic transit time
  alpha = 0.32 # Grubbs exponent
  rho = 0.34 # resting oxygen extraction fraction
  V0 = 0.02 # resting blood volume fraction
  dt = dt / 1000 # [s]

  n = np.shape(signals)[0]

  curr_s = 1e-10 * np.ones((1, n))
  curr_f = 1e-10 * np.ones((1, n))
  curr_v = 1e-10 * np.ones((1, n))
  curr_q = 1e-10 * np.ones((1, n))

  for i in tqdm(range(1, np.shape(signals)[1])):

    new_q = curr_q + dt * (1/tau) * (curr_f/rho * (1-np.power((1-rho), (1/curr_f))) - np.power(curr_v, (1/alpha) - 1) * curr_q)

    new_v = curr_v + dt * (1/tau) * (curr_f - np.power(curr_v, 1/alpha))

    new_f = curr_f + dt * curr_s

    new_s = curr_s + dt * (np.reshape(signals[:, i-1].copy(), (1, n)) - k * curr_s - gamma*(curr_f - 1))

    y = V0 * (7*rho*(1-curr_q) + 2*(1-np.divide(curr_q, curr_v)) + (2*rho-0.2)*(1 - curr_v))

    curr_q = new_q
    curr_v = new_v
    curr_f = new_f
    curr_s = new_s

    BOLD[:, i] = np.reshape(y, (n))

  return BOLD


def bold_rs_pipeline(bold_sig, dt, fs):

  '''
  Takes np.array of BOLD signals with shape [node, step] and treats it for FC analysis.
  Pipeline based on Cabral et al. 2011 Neuroimage and Deco et al 2021 Science Advances

  Args:
    bold_sig (array): array with shape [time step, node] containing BOLD signals for all nodes
    dt (float): time step of original signal (ms)
  Returns:
    processed_sig (array): resampled bold signal
  '''

  ns = bold_sig.shape[-1]
  frs = 1/(0.72)
  fs = 1e3/dt
  nrs = int(fs/frs)
  bdr = int(60 * fs) # Time to remove at the beginning and end of signal, to avoid boundary effects from the Balloon-Windkessel model

  n_samples = int(frs * (bold_sig.shape[-1] - 2 * bdr) / fs)
  treated_sig = np.zeros((bold_sig.shape[0], n_samples))

  for n in range(bold_sig.shape[0]):
      treated_sig[n, :] = sig.resample(bold_sig[n, bdr:-bdr], n_samples)
      treated_sig[n, :] -= np.mean(treated_sig[n, :])

  return treated_sig

def bold_FC_pipeline(bold_sig, dt, fs):

  '''
  Takes np.array of BOLD signals with shape [node, step] and treats it for FC analysis.
  Pipeline based on Cabral et al. 2011 Neuroimage and Deco et al 2021 Science Advances

  Args:
    bold_sig (array): array with shape [time step, node] containing BOLD signals for all nodes
    dt (float): time step of original signal (ms)
  Returns:
    processed_sig (array): resampled bold signal
  '''

  ns = bold_sig.shape[-1]
  frs = 1/(0.72)
  fs = 1e3/dt
  nrs = int(fs/frs)
  bdr = int(60 * fs) # Time to remove at the beginning and end of signal, to avoid boundary effects from the Balloon-Windkessel model

  n_samples = int(frs * (bold_sig.shape[-1] - 2 * bdr) / fs)
  treated_sig = np.zeros((bold_sig.shape[0], n_samples))

  for n in range(bold_sig.shape[0]):
      treated_sig[n, :] = sig.resample(bold_sig[n, bdr:-bdr], n_samples)

  b, a = sig.butter(2, [0.008, 0.08], btype = 'band', output = 'ba', fs = frs)

  treated_sig = sig.filtfilt(b, a, treated_sig, axis = -1)

  return treated_sig

def plot_BOLD(BOLD, n = 10, fs = 0.5):

  '''
  Plots BOLD signal for first n nodes.

  Args:
    BOLD (array): processed BOLD signal with shape [node, sample]
    T_sig (float): duration of recording
    n (int): number of nodes to be shown
    fs (float): sampling frequency of signal
  '''

  plt.figure(figsize=(12, 6))
  plt.plot(np.arange(0, BOLD.shape[-1])/fs, BOLD[:n, :].T + np.r_[:n], color = 'k')
  for i in range(n):
    plt.axhline(y = i, color = 'k', alpha = 0.4)
  plt.yticks([i for i in range(n)], ['Node {}'.format(i) for i in range(n)], fontsize = 14)
  plt.xlabel('Time (s)', fontsize = 14)
  plt.ylim([-1, n])
  plt.title('BOLD Signal', fontsize = 14)
  plt.tight_layout()
  plt.show()

def plot_BOLD_standardized(BOLD, n = 10, fs = 0.5):

  '''
  Plots BOLD signal for first n nodes.

  Args:
    BOLD (array): processed BOLD signal with shape [node, sample]
    T_sig (float): duration of recording
    n (int): number of nodes to be shown
    fs (float): sampling frequency of signal
  '''

  BOLD_norm = BOLD.copy()
  for i in range(BOLD.shape[0]):
    BOLD_norm[i, :] = (BOLD[i, :] - np.mean(BOLD[i, :])) / np.std(BOLD[i, :])

  plt.figure(figsize=(12, 6))
  plt.plot(np.arange(0, BOLD_norm.shape[-1])/fs, BOLD_norm[:n, :].T / 3 + np.r_[:n], color = 'k')
  for i in range(n):
    plt.axhline(y = i, color = 'k', alpha = 0.4)
  plt.yticks([i for i in range(n)], ['Node {}'.format(i) for i in range(n)], fontsize = 14)
  plt.xlabel('Time (s)', fontsize = 14)
  plt.ylim([-1, n])
  plt.title('BOLD Signal - Standardized', fontsize = 14)
  plt.tight_layout()
  plt.show()

# Setup